In [ ]:
import os
os.chdir(os.path.pardir)

import numpy as np
import pandas as pd
import time
import itertools
from itertools import product

from regain.covariance import GraphicalLasso as rg_GL
#from regain import datasets, utils

from sklearn.covariance import GraphicalLasso as sk_GL
from sklearn import set_config

from regain_benchmark import regain_time
from sklearn_benchmark import sklearn_time
from gglasso_benchmark import call_gglasso, gglasso_time

from utilita import network_generation, model_solution, benchmark_parameters 
from utilita import sparsity_benchmark, dict_shape, calc_hamming_dict, benchmarks_dataframe

from plots import plot_accuracy, plot_scalability, plot_lambdas, plot_bm

from tqdm.contrib import tzip
from tqdm.contrib.itertools import product

set_config(print_changed_only=False)
pd.set_option('display.max_columns', None)

## Data

### Power networks

In [ ]:
S_dict=dict()
X_dict=dict()
Theta_dict=dict()

p_list=[100, 500, 1000, 2000, 5000]
N_list=[110, 550, 1100, 2200, 5500]

p_list=[100, 500]
N_list=[110, 550]

print(" Power network generation ".center(40, '-'))

for p, N in tzip(p_list, N_list):
    start = time.perf_counter()
    S, X, Theta = network_generation(p, N, M=10)
    end = time.perf_counter()
    print("p: %5d, N : %5d, Time : %5.4f" % (p, N, end-start))

    S_dict[p, N] = S.copy()
    X_dict[p, N] = X.copy()
    Theta_dict[p, N] = Theta.copy()

In [ ]:
print("\n Shape of S_i:", dict_shape(S_dict))
print("\n Shape of X_i:", dict_shape(X_dict))
print("\n Shape of Theta_i:", dict_shape(Theta_dict))

### Hyperparameters

In [ ]:
lambda_list = [0.5, 0.1, 0.05]
sk_params, rg_params, gglasso_params, lambda_list = benchmark_parameters(lambda_list = lambda_list)

### Model solution

In [ ]:
model_time_dict = dict()
model_Z_dict = dict()
reference_solver = "regain"

print(f"Solving for a reference solution with solver {reference_solver}:")

for X, l1 in product(list(X_dict.values()), lambda_list):
    
    Z, Z_time, info = model_solution(solver=reference_solver, X=X, lambda1=l1)
    
    key = "p_" + str(X.shape[1]) + "_N_" + str(X.shape[0]) + "_l1_" + str(l1)
    model_time_dict.update({key: Z_time})
    model_Z_dict.update({key: Z})

In [ ]:
time_dict = dict()
accuracy_dict = dict()
Z_dict = dict()

n_iter = 3

### GGLasso

In [ ]:
for X, S in tzip(list(X_dict.values()), list(S_dict.values())):
    Omega_0 = np.eye(len(S))
    gg_time, gg_accuracy, Z_gg = gglasso_time(S=S, X=X, Omega_0=Omega_0, Z=model_Z_dict, lambda_list=lambda_list,
                                              n_iter=n_iter, gglasso_params=gglasso_params, warm_start=False)
    
    time_dict.update(gg_time)
    accuracy_dict.update(gg_accuracy)
    Z_dict.update(Z_gg)

### Sklearn

In [ ]:
for X, S in tzip(list(X_dict.values()), list(S_dict.values())):
    sk_time, sk_accuracy, Z_sk = sklearn_time(X=X, Z=model_Z_dict, sk_params=sk_params, lambda_list=lambda_list, \
                                              n_iter=n_iter)
    
    time_dict.update(sk_time)
    accuracy_dict.update(sk_accuracy)
    Z_dict.update(Z_sk)

### Regain

In [ ]:
for X, S in tzip(list(X_dict.values()), list(S_dict.values())):
    rg_time, rg_accuracy, Z_rg = regain_time(X=X, Z=model_Z_dict, rg_params=rg_params, lambda_list=lambda_list, \
                                             n_iter=n_iter, warm_start=False)
    
    time_dict.update(rg_time)
    accuracy_dict.update(rg_accuracy)
    Z_dict.update(Z_rg)

### Hamming distances

In [ ]:
hamming_dict = calc_hamming_dict(Theta_dict=Theta_dict, Z_dict=Z_dict, t_rounding=1e-8)

### Visualization

In [ ]:
df = benchmarks_dataframe(times=time_dict, acc=accuracy_dict, hamming=hamming_dict)
df.head()

In [ ]:
plot_bm(df, lambda_list, min_acc = 5e-3, log_scale = True)

In [ ]:
fig = plot_lambdas(df, upper_bound=0.01, lower_bound=0.000001)
fig.show()

In [ ]:
fig = plot_accuracy(df, upper_bound=0.1, lower_bound=0.0000001, lambda_filter=0.05, sortby=['p', 'time'])
fig.show()

In [ ]:
frames = sparsity_benchmark(df, upper_bound=0.01, lower_bound=0.0001, lambda_filter=0.1)
frames